<a href="https://colab.research.google.com/github/Dawoodayaz20/Gen_AI_Engineering/blob/main/AI_RAG_Proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain-pinecone  langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00


In [ ]:
import getpass
import os
from pinecone import Pinecone, ServerlessSpec

from google.colab import userdata

Pinecone_api_key = userdata.get('pinecone_api')
pc = Pinecone(api_key=Pinecone_api_key)

In [ ]:
import time

index_name='rag-project' # change if desired

# pc.create_index(
#       name=index_name,
#       dimension=768,
#       metric='cosine',
#       spec=ServerlessSpec(cloud='aws', region='us-east-1')
# )

index = pc.Index(index_name)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

os.environ["GOOGLE_API_KEY"] = userdata.get('gemini_key')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("Hello Jarvis")
vector

[0.021213820204138756,
 -0.03951342776417732,
 -0.04750613495707512,
 -0.011852039955556393,
 0.02909448929131031,
 -0.03918665647506714,
 0.01114931795746088,
 -0.015147551894187927,
 -0.023513514548540115,
 0.03961820900440216,
 0.01858181320130825,
 0.02740069106221199,
 -0.019654536619782448,
 -0.02854929119348526,
 0.02185327745974064,
 -0.008514615707099438,
 0.02271430194377899,
 0.011460842564702034,
 -0.004811845254153013,
 -0.043977364897727966,
 0.021124470978975296,
 0.03570636734366417,
 -0.006520173978060484,
 0.016207147389650345,
 0.011258025653660297,
 -0.014700954779982567,
 0.010814188979566097,
 -0.07401204109191895,
 0.01287948340177536,
 0.07723018527030945,
 -0.050553854554891586,
 0.06308408826589584,
 -0.057293009012937546,
 0.002807562705129385,
 0.026968933641910553,
 -0.0358317606151104,
 0.04582088440656662,
 0.022589391097426414,
 -0.009656926617026329,
 0.003717352170497179,
 0.0272509828209877,
 -0.023445790633559227,
 0.0015217209002003074,
 0.015577365

Getting the first 5 lines of our output.

In [ ]:
vector[:5]

[0.021213820204138756,
 -0.03951342776417732,
 -0.04750613495707512,
 -0.011852039955556393,
 0.02909448929131031]

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="The weather was very hot last day",
    metadata={"source": "news"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

documents = [
    document_1,
    document_2,
    document_3,
]

uuids = [str(uuid4()) for _ in range (len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)


['04c94eb3-37d0-4aec-a766-2536060a0f04',
 '3020e204-4098-4ad4-bdff-a7f33f984821',
 '5aab2c1d-8430-4d83-8091-034a674ebcf0']

Data retriever

In [ ]:
results = vector_store.similarity_search(
    "Stock Market",
    k=2
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* The stock market is down 500 points today due to fears of a recession. [{'source': 'news'}]
* The stock market is down 500 points today due to fears of a recession. [{'source': 'news'}]


In [ ]:
results = vector_store.similarity_search_with_score(
    "News about Stocks"
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.652291] The stock market is down 500 points today due to fears of a recession. [{'source': 'news'}]
* [SIM=0.652291] The stock market is down 500 points today due to fears of a recession. [{'source': 'news'}]
* [SIM=0.652291] The stock market is down 500 points today due to fears of a recession. [{'source': 'news'}]
* [SIM=0.543508] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


Function for the project

In [ ]:
def answer_to_user(query: str):
  #Vector Search
  results = vector_store.similarity_search(query, k=2)
  # Pass to Model Vector Results + User query
  final_answer = model(results, query)
  return final_answer